In [1]:
#Import used
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns
import math
import multiprocessing as mp
from functools import partial

np.set_printoptions(precision=3, suppress=True)

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.12.0


In [3]:
dataset = pd.read_csv("Cleaned_data_v2/cleaned_car_data_v2.csv", sep=',')
dataset = dataset[dataset['mileage'] >= 15000]
dataset

C:\Users\ethan\AppData\Local\Temp\ipykernel_16124\3860652569.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("Cleaned_data_v2/cleaned_car_data_v2.csv", sep=',')


,price,body_type,exterior_color,has_accidents,listed_date,make_name,mileage,model_name,owner_count,year,dealer_zip,major_options,trim_name,wheel_system
0,200.0,SUV / Crossover,BLUE,False,2020-07-07,Ford,150000.0,Explorer,4.0,2005,33178,[3],XLT V6,RWD
1,249.0,Sedan,Cashmere Metallic,True,2020-08-26,Buick,202158.0,Century,5.0,2005,49684,[3],Custom Sedan FWD,FWD
2,347.0,Sedan,Titanium Gray Metallic,False,2020-07-16,Mazda,213512.0,MAZDA3,1.0,2006,84107,[3],i,FWD
3,395.0,SUV / Crossover,Black,False,2020-05-02,Jeep,122610.0,Liberty,3.0,2002,44256,[3],Sport 4WD,4WD
4,400.0,Sedan,SILVER,True,2016-07-26,Oldsmobile,111111.0,Alero,1.0,2002,29379,[3],GX,FWD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528589,999995.0,Sedan,Tuxedo Black Metallic,False,2020-09-11,Ford,80306.0,Fusion,2.0,2014,78257,"[3, 4]",SE,FWD
2528594,1199500.0,Hatchback,Dk. Gray,False,2020-08-16,Chevrolet,123633.0,Volt,2.0,2015,92335,"[3, 4, 6]",Premium FWD,FWD
2528601,1599500.0,Minivan,Black,False,2020-07-27,Dodge,81992.0,Grand Caravan,2.0,2017,72764,"[3, 4, 37, 6, 8, 9, 60, 61]",GT FWD,FWD
2528602,1750000.0,Sedan,Black,False,2017-09-08,BMW,121043.0,7 Series,4.0,1996,89139,"[3, 60, 61, 52]",750iL RWD,RWD


In [4]:
def one_hot(x):
    x = x.strip("[] ").split(",")
    #print(x)
    temp = [0] * 77
    for a in x:
        temp[int(a)] = 1
    #print(temp)
    return temp

one_hot_col_lst = []
for a in range(77):
    one_hot_col_lst.append(str(a))



In [5]:
#%%timeit



#pd.get_dummies(dataset, columns=['wheel_system'], prefix='', prefix_sep='')#260ms

#dataset['has_accidents'] = dataset.apply(lambda row: 1 if row['has_accidents'] else 0, axis=1)
#4 sec
#dataset['has_accidents'] = dataset['has_accidents'].astype(bool).astype(int)

In [6]:
#Sets accidents to either 0 or 1 depending on if the column value was True or False
dataset['has_accidents'] = dataset['has_accidents'].astype(bool).astype(int)#One hot encodes each of the following columns 
dataset = pd.get_dummies(dataset, columns=['wheel_system'], prefix='', prefix_sep='')
dataset = pd.get_dummies(dataset, columns=['make_name'], prefix='', prefix_sep='')
dataset = pd.get_dummies(dataset, columns=['body_type'], prefix='', prefix_sep='')
##for i in range(1,77):
##    print(i)
##    dataset[i] = [0]*len(dataset)
##    dataset[i] = dataset.apply(lambda row: 1 if str(i) in row['major_options'].strip('][').split(', ') else 0, axis=1)
dataset['major_options'] = dataset['major_options'].apply(one_hot)
dataset[one_hot_col_lst] =pd.DataFrame(dataset.pop('major_options').values.tolist(), index=dataset.index)
#Removes the columns that aren't currently being used
#Color and Major options can eventually be included, but would prefer them to be more simplfied
dataset = dataset.drop(columns=['exterior_color', 'model_name', 'listed_date', 'dealer_zip', 'trim_name'])
dataset

,price,has_accidents,mileage,owner_count,year,4WD,4X2,AWD,FWD,RWD,...,67,68,69,70,71,72,73,74,75,76
0,200.0,0,150000.0,4.0,2005,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,249.0,1,202158.0,5.0,2005,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,347.0,0,213512.0,1.0,2006,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,395.0,0,122610.0,3.0,2002,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,400.0,1,111111.0,1.0,2002,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528589,999995.0,0,80306.0,2.0,2014,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2528594,1199500.0,0,123633.0,2.0,2015,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2528601,1599500.0,0,81992.0,2.0,2017,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2528602,1750000.0,0,121043.0,4.0,1996,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
dataset = dataset.sample(frac=1)
dataset

,price,has_accidents,mileage,owner_count,year,4WD,4X2,AWD,FWD,RWD,...,67,68,69,70,71,72,73,74,75,76
663513,18490.0,0,73171.0,2.0,2015,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
38220,5499.0,0,163298.0,3.0,2008,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
842881,20579.0,0,18873.0,2.0,2018,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2201852,46437.0,0,37174.0,1.0,2019,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
619570,17980.0,0,55193.0,2.0,2017,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110657,23988.0,0,25061.0,1.0,2017,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
849847,20695.0,0,47216.0,1.0,2019,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1604184,30885.0,1,84905.0,2.0,2013,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
29308,4995.0,0,143000.0,7.0,2013,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def plot_loss(history, lr):
  plt.plot(history['loss'], label=f"loss {lr}")
  plt.plot(history['val_loss'], label=f"val_loss {lr}")
  print(min(history['loss']), max(history['loss']))
  plt.ylim([min(min(history['val_loss']), min(history['loss'])), max(max(history['loss']), max(history['val_loss']))])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Price]')
  plt.title(f"Runs for LR: {lr}")
  plt.legend()
  plt.grid(True)

In [9]:
test_results = {}
hist_results = {}

In [10]:
learningRates = [0.01, 0.025, 0.05, 0.1]
cutOff = math.floor(len(dataset) / 5)
for lr in learningRates:
    print(f"Testing Learning Rate: {lr}")
    errorAvgs = []
    for i in range(0,5):
        print(f"Tetsing the {i+1}th fifth")
        train_dataset = dataset.iloc[i*cutOff:cutOff*(i+1)]
        test_dataset = dataset.drop(train_dataset.index)

        train_features = train_dataset.copy()
        test_features = test_dataset.copy()

        train_labels = train_features.pop('price')
        test_labels = test_features.pop('price')

        model_params = np.array(train_features)
        #Creates the normalizer for the parameters
        normalizer = tf.keras.layers.Normalization(input_shape=[len(train_dataset.columns) - 1,], axis=-1)
        normalizer.adapt(model_params) 

        linear_model = tf.keras.Sequential([
            normalizer,
            layers.Dense(units=16),
            layers.Dense(units=32),
            layers.Dense(units=5)
        ])

        linear_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss='mean_absolute_error')
        
        history = linear_model.fit(
            train_features,
            train_labels,
            epochs=25,
            # Suppress logging.
            verbose=0,
            # Calculate validation results on 20% of the training data.
            validation_split = 0.2)
        
        hist = pd.DataFrame(history.history)
        if lr in hist_results.keys():
            hist_results[lr] += hist
        else:
            hist_results[lr] = hist

        error = linear_model.evaluate(
            test_features,
            test_labels, verbose=0)

        print(f"Error for LR = {lr} on {1+i}th is: {error}")

        errorAvgs.append(error)
        
    avgHist = hist_results[lr] / 5
    avgHist['epoch'] = history.epoch
    plot_loss(avgHist, lr)
    test_results[lr] = (np.mean(errorAvgs), errorAvgs)

test_results

Testing Learning Rate: 0.01
Tetsing the 1th fifth


KeyboardInterrupt: 

In [ ]:
test_results
for key in test_results.keys():
    print(f"Error for Learning Rate {key} is {test_results[key][0]}")

In [ ]:
plot_loss(hist_results[0.1] / 5, 0.1)